In [126]:
import torch_geometric as TG
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv

import numpy as np
import networkx as nx
import torch_geometric as TG
import torch
import pickle as pkl

import torch.optim as optim


In [388]:
experiments = []
adjacency_matrices = []

#做n_experiments次传播实验
n_experiments = 10
n_time_steps = 30
immune_prob = 0.1
for i in range(n_experiments):
    #每次实验一个不同的网络
    graph = nx.Graph()
    n_nodes = np.random.choice(99)+1 #尺寸不同的网络
    # Chain network
    #lst = np.array(range(n_nodes - 1))
    #lst1 = lst + 1
    #edges = list(zip(lst,lst1))
    #graph.add_edges_from(edges)
    graph = nx.erdos_renyi_graph(n_nodes, 0.1)
    
    vector = np.zeros(n_nodes)
    for sd in range(2):
        rnd_nd = np.random.choice(n_nodes)
        vector[rnd_nd]=1
    temp = np.array([np.ones(vector.shape[0])-vector,vector]).transpose()
    adj = nx.adjacency_matrix(graph).todense() + np.eye(n_nodes) #加上了自环，否则GCN无法正确判断一个被感染的节点始终被感染
    states = [temp]
    vec = vector

    for step in range(n_time_steps):
        # 一次传播，即用邻接矩阵adj乘以向量vec
        next_new = np.sign(np.array(adj.dot(vec))) * np.random.rand(len(vec))
        
        bools = next_new > 0.5
        next_new = np.array(bools)
        vec = np.array(np.sign(next_new + vec))[0]
        # 将状态转为one hot编码，即1-->[0,1]; 0-->[1,0]
        temp = np.array([np.ones(vec.shape[0])-vec,vec]).transpose()
        states.append(temp)
    experiments.append(states)
    adjacency_matrices.append(adj)


In [389]:
class Net(nn.Module):
    def __init__(self, h_dim):
        super(Net, self).__init__()

        self.emb = nn.Linear(2, h_dim)
        self.gnn = GCNConv(h_dim, h_dim, bias=False)
        self.pred= nn.Linear(h_dim, 2)
        self.criterion = nn.NLLLoss()

        self.opt = torch.optim.Adam(self.parameters(), lr=1E-3)

    def forward(self, x, edge_index):
        x = self.emb(x)
        x = self.gnn(x, edge_index)
        x = F.relu(x)
        y_hat = F.log_softmax(self.pred(x),dim=1)

        return y_hat
    
    

In [390]:
train_x = []
train_y = []
train_adj = []
for i in range(n_experiments-1):
    train_x.extend(experiments[i][:-1])
    train_y.extend(experiments[i][1:])
    train_adj.append(adjacency_matrices[i])
test_x = experiments[-1][:-1]
test_y = experiments[-1][1:]
test_adj = adjacency_matrices[-1]


In [391]:
hidden_sz = 32
net = Net(hidden_sz)


opt = optim.Adam(net.parameters(), lr=0.001)
epochs = 5000
for i in range(epochs):
    ridx = np.random.choice(len(train_x))
    data_x = train_x[ridx]
    data_y = train_y[ridx]
    experiment_idx = ridx // n_time_steps
    edgelist = torch.nonzero(torch.Tensor(train_adj[experiment_idx])).t()
    opt.zero_grad()
    #print(ridx)
    #print(data_x.shape)
    hat=net(torch.Tensor(data_x),edgelist)
    #print(np.nonzero(data_x[:,1]))
    #print(torch.nonzero(torch.sign(torch.exp(hat)[:,1]-0.5)+1).view(-1))
    #print(np.nonzero(data_y[:,1]))
    loss = net.criterion(hat, torch.LongTensor(data_y[:,1]))
    loss.backward()
    opt.step()
    print(loss)
    

tensor(0.7337, grad_fn=<NllLossBackward>)
tensor(0.5907, grad_fn=<NllLossBackward>)
tensor(0.5334, grad_fn=<NllLossBackward>)
tensor(1.0047, grad_fn=<NllLossBackward>)
tensor(0.5147, grad_fn=<NllLossBackward>)
tensor(0.5533, grad_fn=<NllLossBackward>)
tensor(0.4978, grad_fn=<NllLossBackward>)
tensor(0.7748, grad_fn=<NllLossBackward>)
tensor(0.5261, grad_fn=<NllLossBackward>)
tensor(0.9230, grad_fn=<NllLossBackward>)
tensor(0.4626, grad_fn=<NllLossBackward>)
tensor(0.4544, grad_fn=<NllLossBackward>)
tensor(0.4574, grad_fn=<NllLossBackward>)
tensor(0.4381, grad_fn=<NllLossBackward>)
tensor(0.4397, grad_fn=<NllLossBackward>)
tensor(0.9806, grad_fn=<NllLossBackward>)
tensor(0.7182, grad_fn=<NllLossBackward>)
tensor(0.4199, grad_fn=<NllLossBackward>)
tensor(0.4130, grad_fn=<NllLossBackward>)
tensor(0.6367, grad_fn=<NllLossBackward>)
tensor(0.6908, grad_fn=<NllLossBackward>)
tensor(0.4108, grad_fn=<NllLossBackward>)
tensor(0.3833, grad_fn=<NllLossBackward>)
tensor(0.5299, grad_fn=<NllLossBac

tensor(0.0390, grad_fn=<NllLossBackward>)
tensor(0.0135, grad_fn=<NllLossBackward>)
tensor(0.0380, grad_fn=<NllLossBackward>)
tensor(0.1515, grad_fn=<NllLossBackward>)
tensor(0.0131, grad_fn=<NllLossBackward>)
tensor(0.0106, grad_fn=<NllLossBackward>)
tensor(0.1345, grad_fn=<NllLossBackward>)
tensor(0.0106, grad_fn=<NllLossBackward>)
tensor(0.0128, grad_fn=<NllLossBackward>)
tensor(0.0346, grad_fn=<NllLossBackward>)
tensor(0.0086, grad_fn=<NllLossBackward>)
tensor(0.0126, grad_fn=<NllLossBackward>)
tensor(0.0092, grad_fn=<NllLossBackward>)
tensor(0.3840, grad_fn=<NllLossBackward>)
tensor(0.0108, grad_fn=<NllLossBackward>)
tensor(0.3130, grad_fn=<NllLossBackward>)
tensor(0.0085, grad_fn=<NllLossBackward>)
tensor(0.0084, grad_fn=<NllLossBackward>)
tensor(0.0093, grad_fn=<NllLossBackward>)
tensor(0.4787, grad_fn=<NllLossBackward>)
tensor(0.0070, grad_fn=<NllLossBackward>)
tensor(0.0068, grad_fn=<NllLossBackward>)
tensor(0.0072, grad_fn=<NllLossBackward>)
tensor(0.0619, grad_fn=<NllLossBac

tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0290, grad_fn=<NllLossBackward>)
tensor(0.0018, grad_fn=<NllLossBackward>)
tensor(0.1691, grad_fn=<NllLossBackward>)
tensor(0.0326, grad_fn=<NllLossBackward>)
tensor(0.3869, grad_fn=<NllLossBackward>)
tensor(0.2962, grad_fn=<NllLossBackward>)
tensor(0.0755, grad_fn=<NllLossBackward>)
tensor(0.0335, grad_fn=<NllLossBackward>)
tensor(0.0017, grad_fn=<NllLossBackward>)
tensor(0.0029, grad_fn=<NllLossBackward>)
tensor(0.0015, grad_fn=<NllLossBackward>)
tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0068, grad_fn=<NllLossBackward>)
tensor(0.0294, grad_fn=<NllLossBackward>)
tensor(0.4953, grad_fn=<NllLossBackward>)
tensor(0.0028, grad_fn=<NllLossBackward>)
tensor(0.0279, grad_fn=<NllLossBackward>)
tensor(0.0024, grad_fn=<NllLossBackward>)
tensor(0.1342, grad_fn=<NllLossBackward>)
tensor(0.2145, grad_fn=<NllLossBackward>)
tensor(0.0026, grad_fn=<NllLossBac

tensor(0.0026, grad_fn=<NllLossBackward>)
tensor(0.0067, grad_fn=<NllLossBackward>)
tensor(0.0014, grad_fn=<NllLossBackward>)
tensor(0.0349, grad_fn=<NllLossBackward>)
tensor(0.0345, grad_fn=<NllLossBackward>)
tensor(0.2133, grad_fn=<NllLossBackward>)
tensor(0.0064, grad_fn=<NllLossBackward>)
tensor(0.1193, grad_fn=<NllLossBackward>)
tensor(0.0014, grad_fn=<NllLossBackward>)
tensor(0.0298, grad_fn=<NllLossBackward>)
tensor(0.2161, grad_fn=<NllLossBackward>)
tensor(0.0027, grad_fn=<NllLossBackward>)
tensor(0.0025, grad_fn=<NllLossBackward>)
tensor(0.0251, grad_fn=<NllLossBackward>)
tensor(0.3831, grad_fn=<NllLossBackward>)
tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0017, grad_fn=<NllLossBackward>)
tensor(0.0213, grad_fn=<NllLossBackward>)
tensor(0.0018, grad_fn=<NllLossBackward>)
tensor(0.0018, grad_fn=<NllLossBackward>)
tensor(0.5262, grad_fn=<NllLossBackward>)
tensor(0.0045, grad_fn=<NllLossBackward>)
tensor(0.0027, grad_fn=<NllLossBackward>)
tensor(0.0044, grad_fn=<NllLossBac

tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0247, grad_fn=<NllLossBackward>)
tensor(0.0053, grad_fn=<NllLossBackward>)
tensor(0.0027, grad_fn=<NllLossBackward>)
tensor(0.0205, grad_fn=<NllLossBackward>)
tensor(0.0228, grad_fn=<NllLossBackward>)
tensor(0.1307, grad_fn=<NllLossBackward>)
tensor(0.0020, grad_fn=<NllLossBackward>)
tensor(0.1931, grad_fn=<NllLossBackward>)
tensor(0.0028, grad_fn=<NllLossBackward>)
tensor(0.0021, grad_fn=<NllLossBackward>)
tensor(0.0030, grad_fn=<NllLossBackward>)
tensor(0.0022, grad_fn=<NllLossBackward>)
tensor(0.0023, grad_fn=<NllLossBackward>)
tensor(0.0023, grad_fn=<NllLossBackward>)
tensor(0.4046, grad_fn=<NllLossBackward>)
tensor(0.0022, grad_fn=<NllLossBackward>)
tensor(0.0032, grad_fn=<NllLossBackward>)
tensor(0.0162, grad_fn=<NllLossBackward>)
tensor(0.0023, grad_fn=<NllLossBackward>)
tensor(0.5014, grad_fn=<NllLossBackward>)
tensor(0.0046, grad_fn=<NllLossBackward>)
tensor(0.0026, grad_fn=<NllLossBackward>)
tensor(0.0160, grad_fn=<NllLossBac

...

tensor(0.0007, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)
tensor(0.4266, grad_fn=<NllLossBackward>)
tensor(0.0018, grad_fn=<NllLossBackward>)
tensor(0.2160, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0096, grad_fn=<NllLossBackward>)
tensor(0.3054, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0100, grad_fn=<NllLossBackward>)
tensor(0.0022, grad_fn=<NllLossBackward>)
tensor(0.0023, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0132, grad_fn=<NllLossBackward>)
tensor(0.0606, grad_fn=<NllLossBackward>)
tensor(0.0003, grad_fn=<NllLossBackward>)
tensor(0.0003, grad_fn=<NllLossBackward>)
tensor(0.1800, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0026, grad_fn=<NllLossBackward>)
tensor(0.0003, grad_fn=<NllLossBac

tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0006, grad_fn=<NllLossBackward>)
tensor(0.0006, grad_fn=<NllLossBackward>)
tensor(0.3394, grad_fn=<NllLossBackward>)
tensor(0.0071, grad_fn=<NllLossBackward>)
tensor(0.0017, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0083, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.4331, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.1950, grad_fn=<NllLossBackward>)
tensor(0.0085, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0079, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0030, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0063, grad_fn=<NllLossBackward>)
tensor(0.1961, grad_fn=<NllLossBackward>)
tensor(0.0064, grad_fn=<NllLossBackward>)
tensor(0.0062, grad_fn=<NllLossBackward>)
tensor(0.1709, grad_fn=<NllLossBac

tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0030, grad_fn=<NllLossBackward>)
tensor(0.1797, grad_fn=<NllLossBackward>)
tensor(0.0006, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0007, grad_fn=<NllLossBackward>)
tensor(0.0033, grad_fn=<NllLossBackward>)
tensor(0.0037, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0011, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.3188, grad_fn=<NllLossBackward>)
tensor(0.0045, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBac

tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0087, grad_fn=<NllLossBackward>)
tensor(0.0007, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0062, grad_fn=<NllLossBackward>)
tensor(0.0007, grad_fn=<NllLossBackward>)
tensor(0.0097, grad_fn=<NllLossBackward>)
tensor(0.3483, grad_fn=<NllLossBackward>)
tensor(0.0007, grad_fn=<NllLossBackward>)
tensor(0.0007, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0006, grad_fn=<NllLossBackward>)
tensor(0.0177, grad_fn=<NllLossBackward>)
tensor(0.3333, grad_fn=<NllLossBackward>)
tensor(0.0021, grad_fn=<NllLossBackward>)
tensor(0.1480, grad_fn=<NllLossBackward>)
tensor(0.0021, grad_fn=<NllLossBackward>)
tensor(0.0092, grad_fn=<NllLossBackward>)
tensor(0.0004, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0460, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBac

tensor(0.0015, grad_fn=<NllLossBackward>)
tensor(0.0015, grad_fn=<NllLossBackward>)
tensor(0.0014, grad_fn=<NllLossBackward>)
tensor(0.0078, grad_fn=<NllLossBackward>)
tensor(0.1186, grad_fn=<NllLossBackward>)
tensor(0.0717, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0015, grad_fn=<NllLossBackward>)
tensor(0.0013, grad_fn=<NllLossBackward>)
tensor(0.0938, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)
tensor(0.0701, grad_fn=<NllLossBackward>)
tensor(0.0017, grad_fn=<NllLossBackward>)
tensor(0.0059, grad_fn=<NllLossBackward>)
tensor(0.0058, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0388, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)
tensor(0.0052, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBac

In [397]:
#在测试集上测试
for xx,yy in zip(test_x,test_y):
    edgelist = torch.nonzero(torch.Tensor(test_adj)).t()
    hat=net(torch.Tensor(xx), edgelist)
    hat = (torch.sign(torch.exp(hat)[:,1]-0.5)+1)/2
    #print(xx[:,1])
    #print(yy[:,1])
    #print(hat)
    yy = torch.Tensor(yy[:,1])
    tp = hat * yy
    p = torch.sum(tp) / (torch.sum(hat)+1e-20)
    r = torch.sum(tp) / (torch.sum(yy)+1e-20)
    f1 = 2*p*r/(p+r+1e-20)
    print(p,r,f1)


tensor(0.6667, grad_fn=<DivBackward0>) tensor(0.6667, grad_fn=<DivBackward0>) tensor(0.6667, grad_fn=<DivBackward0>)
tensor(0.6111, grad_fn=<DivBackward0>) tensor(0.9167, grad_fn=<DivBackward0>) tensor(0.7333, grad_fn=<DivBackward0>)
tensor(0.8000, grad_fn=<DivBackward0>) tensor(0.9524, grad_fn=<DivBackward0>) tensor(0.8696, grad_fn=<DivBackward0>)
tensor(0.7500, grad_fn=<DivBackward0>) tensor(1., grad_fn=<DivBackward0>) tensor(0.8571, grad_fn=<DivBackward0>)
tensor(0.9062, grad_fn=<DivBackward0>) tensor(1., grad_fn=<DivBackward0>) tensor(0.9508, grad_fn=<DivBackward0>)
tensor(0.8824, grad_fn=<DivBackward0>) tensor(1., grad_fn=<DivBackward0>) tensor(0.9375, grad_fn=<DivBackward0>)
tensor(0.9412, grad_fn=<DivBackward0>) tensor(1., grad_fn=<DivBackward0>) tensor(0.9697, grad_fn=<DivBackward0>)
tensor(0.9706, grad_fn=<DivBackward0>) tensor(1., grad_fn=<DivBackward0>) tensor(0.9851, grad_fn=<DivBackward0>)
tensor(0.9706, grad_fn=<DivBackward0>) tensor(1., grad_fn=<DivBackward0>) tensor(0.9